In [1]:
class Event:
	class_name: str
	instance_name: str
	has_agent: list[str]
	has_object: list[str]

Leave-One-Out


In [ ]:
from typing import List, Callable, Tuple, Any

def best_similarity(
    A: List[Any],
    B: List[Any],
    sim: Callable[[Any, Any], float],
    penalty: float
) -> Tuple[float, List[Tuple[int, int]]]:
    """
    Devuelve:
    - score óptimo
    - lista de emparejamientos (i, j), índices en A y B
      Los eventos no usados no aparecen en la lista
    """

    n, m = len(A), len(B)

    # DP y backtracking
    dp = [[0.0] * (m + 1) for _ in range(n + 1)]
    bt = [[None] * (m + 1) for _ in range(n + 1)]
    
    # dp[i][j] La mejor similitud total posible usando los primeros i eventos de la lista A y los primeros j eventos de la lista B.
    # dp[0][0] → no usar ningún evento
    # dp[i][0] → usar i eventos de A y ninguno de B
    # dp[n][m] → solución óptima final

    # Casos base
    for i in range(1, n + 1):
        dp[i][0] = -i * penalty
        bt[i][0] = ('A', i - 1)

    for j in range(1, m + 1):
        dp[0][j] = -j * penalty
        bt[0][j] = ('B', j - 1)

    # DP principal
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = dp[i - 1][j - 1] + sim(A[i - 1], B[j - 1])
            skip_a = dp[i - 1][j] - penalty
            skip_b = dp[i][j - 1] - penalty

            best = max(match, skip_a, skip_b)
            dp[i][j] = best

            if best == match:
                bt[i][j] = ('M', i - 1, j - 1)
            elif best == skip_a:
                bt[i][j] = ('A', i - 1)
            else:
                bt[i][j] = ('B', j - 1)

    # Reconstrucción
    i, j = n, m
    matches = []

    while i > 0 or j > 0:
        step = bt[i][j]

        if step[0] == 'M':
            _, ai, bj = step
            matches.append((ai, bj))
            i -= 1
            j -= 1
        elif step[0] == 'A':
            i -= 1
        else:  # 'B'
            j -= 1

    matches.reverse()
    return dp[n][m], matches


In [ ]:
A = [1, 4, 7]
B = [2, 6]

def sim(a, b):
    return -abs(a - b)

penalty = 2

score, matches = best_similarity(A, B, sim, penalty)

print("Score óptimo:", score)
print("Emparejamientos:")
for a, b in matches:
    print(f"{A[a]} ↔ {B[b]} (sim={sim(A[a],B[b])})")

Score óptimo: -4.0
Emparejamientos:
1 ↔ 2 (sim=-1)
7 ↔ 6 (sim=-1)
